In [ ]:
import os

from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings

In [ ]:
# Option 1: use an OpenAI account
# openai_api_key: str = "PLACEHOLDER FOR YOUR API KEY"
# openai_api_version: str = "2023-05-15"
# model: str = "text-embedding-ada-002"

import openai

API_SECRET_KEY = "sk-r0WeYOdkMjzYdnSxEcC8B931Aa904e4bBaCcAc2a57D803F1"
BASE_URL = "https://svip.xty.app/v1"
os.environ["OPENAI_API_KEY"] = API_SECRET_KEY
os.environ["OPENAI_API_BASE"] = BASE_URL
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os

# 设置 OpenAI API 密钥
# os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

# 加载文档
loader = TextLoader("term.txt")
documents = loader.load()

# 分割文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# 使用 HuggingFace 的 sentence-transformers/all-mpnet-base-v2 作为 embedding 模型
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# 创建 Chroma 数据库
db = Chroma.from_documents(texts, embeddings)

# 使用 similarity_search_with_relevance_scores 进行检索
query = "特别说明：针对有些专家学者对《意见》中的两处疑问，编者专门联系《意见》起草者"
results = db.similarity_search_with_relevance_scores(query, k=5)
 
# 打印检索结果
for doc, score in results:
    print(f"Content: {doc.page_content}")
    print(f"Relevance Score: {score}")
    print("-----")

In [ ]:
# 使用 GPT-4 进行生成
llm = OpenAI(model_name="gpt-4")
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

# 提问并获取回答
query = "your_query"
result = qa_chain.run(query)
print(f"Answer: {result}")
